In [1]:
! pip install tensorflow

In [4]:
!pip install symspellpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 11.9 MB/s eta 0:00:00


In [12]:
# Download GloVe 6B embeddings (100D is small and good for Colab)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

# Load 100D GloVe
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs



--2025-06-04 05:36:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-06-04 05:36:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-06-04 05:36:47--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
import os
import urllib.request

dictionary_path = "frequency_dictionary_en_82_765.txt"
if not os.path.exists(dictionary_path):
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/frequency_dictionary_en_82_765.txt",
        dictionary_path
    )


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


In [17]:
df = pd.read_csv('/content/output.csv')

print("Columns:", df.columns)
print("Sample data:\n", df.head())
print("Shape:", df.shape)

if 'text' not in df.columns or 'label' not in df.columns:
    raise ValueError("CSV must have 'text' and 'label' columns")

df = df[['text', 'label']]
df.dropna(inplace=True)
print("After dropping NaNs shape:", df.shape)
print("Labels before mapping:", df['label'].unique())

label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label'] = df['label'].map(label_map)
print("Labels after mapping:", df['label'].unique())

df = df.dropna(subset=['label'])
print("After dropping NaN labels shape:", df.shape)


Columns: Index(['Unnamed: 0', 'text', 'sentiment', 'label'], dtype='object')
Sample data:
    Unnamed: 0                                               text sentiment  \
0           0  module , ek program hoti hai , jismen ya to so...   neutral   
1           1  aur hamne aume samood ke pas unke bhaee saleh ...  positive   
2           2  aur jab unhen yad dilaya jata hai , to ve yad ...   neutral   
3           3            tumhen २०११ ka ted prize mil gaya hai\n  positive   
4           4  unhonne bad science karne ke lie ye delhi univ...   neutral   

   label  
0      1  
1      2  
2      1  
3      2  
4      1  
Shape: (2766, 4)
After dropping NaNs shape: (2766, 2)
Labels before mapping: [1 2 0]
Labels after mapping: [nan]
After dropping NaN labels shape: (0, 2)


In [19]:
import pandas as pd
import numpy as np
import re
import string
import os
import urllib.request
import pickle
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout, Layer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from symspellpy import SymSpell, Verbosity

# Download SymSpell dictionary if not already downloaded
dictionary_path = "frequency_dictionary_en_82_765.txt"
if not os.path.exists(dictionary_path):
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/frequency_dictionary_en_82_765.txt",
        dictionary_path
    )

# Initialize SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Spell correction using SymSpell
def correct_spelling(text):
    corrected_words = []
    for word in text.split():
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        corrected_words.append(suggestions[0].term if suggestions else word)
    return ' '.join(corrected_words)

# Text cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = correct_spelling(text)
    return text

# Load dataset
df = pd.read_csv('/content/output.csv')

# Select relevant columns
df = df[['text', 'label']]
df.dropna(inplace=True)

print("Shape after dropping NaNs:", df.shape)
print("Unique labels:", df['label'].unique())

# Labels are already numeric, convert to int
df['label'] = df['label'].astype(int)

# Clean text
df['cleaned'] = df['text'].apply(clean_text)

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned'])

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

sequences = tokenizer.texts_to_sequences(df['cleaned'])
padded = pad_sequences(sequences, maxlen=100, padding='post')

# Load GloVe embeddings (make sure glove.6B.100d.txt is downloaded and available)
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((10000, embedding_dim))

for word, i in word_index.items():
    if i < 10000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    padded, df['label'], test_size=0.2, random_state=42
)

print("y_train unique values:", np.unique(y_train))
print("y_test unique values:", np.unique(y_test))

# Custom Attention Layer to fix KerasTensor error
class SimpleAttention(Layer):
    def __init__(self, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)

    def call(self, inputs):
        # inputs shape: (batch_size, timesteps, features)
        score = tf.nn.softmax(inputs, axis=1)  # softmax on timesteps axis
        context_vector = tf.reduce_sum(score * inputs, axis=1)
        return context_vector

# Build model
input_layer = Input(shape=(100,))
embedding_layer = Embedding(
    input_dim=10000,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False
)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention_output = SimpleAttention()(lstm_layer)
dropout_layer = Dropout(0.5)(attention_output)
output_layer = Dense(3, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[early_stop], batch_size=64)

# Ensure models directory exists
os.makedirs('models', exist_ok=True)
model.save('models/lstm_model.h5')

# Evaluate
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))


Shape after dropping NaNs: (2766, 2)
Unique labels: [1 2 0]
y_train unique values: [0 1 2]
y_test unique values: [0 1 2]
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 294ms/step - accuracy: 0.4177 - loss: 1.0738 - val_accuracy: 0.5756 - val_loss: 0.9432
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - accuracy: 0.5409 - loss: 0.9756 - val_accuracy: 0.5914 - val_loss: 0.9141
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 242ms/step - accuracy: 0.5418 - loss: 0.9648 - val_accuracy: 0.6005 - val_loss: 0.9057
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 189ms/step - accuracy: 0.5390 - loss: 0.9342 - val_accuracy: 0.5914 - val_loss: 0.9009
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 191ms/step - accuracy: 0.5663 - loss: 0.9370 - val_accuracy: 0.6050 - val_loss: 0.8813
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 278ms/step - accuracy: 0.5863 - loss: 0.8961 - val_accuracy: 0.6117 - val_loss: 0.8748
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 181ms/step - accuracy: 0.5868 - loss: 0.8772 - val_accuracy: 0.6117

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step
              precision    recall  f1-score   support

           0       0.45      0.15      0.22       121
           1       0.62      0.67      0.64       238
           2       0.46      0.61      0.53       195

    accuracy                           0.53       554
   macro avg       0.51      0.48      0.46       554
weighted avg       0.53      0.53      0.51       554



In [ ]:
def predict_sentiment(text, tokenizer, model, max_length=100):
    # Preprocess input text if needed (lowercase, remove punctuation etc.) based on your original pipeline
    text = text.lower()
    text = ''.join([c for c in text if c not in string.punctuation])

    # Tokenize and pad the text
    sequences = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

    # Predict sentiment
    prediction = model.predict(padded)

    # Assuming binary sentiment classification (0=negative,1=positive)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

    return sentiment

# Example usage:
while True:
    user_input = input("Enter text to analyze sentiment (or 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    sentiment = predict_sentiment(user_input, tokenizer, model)
    print(f"Sentiment: {sentiment}")


Enter text to analyze sentiment (or 'exit' to quit): ye product acha nahi hai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Sentiment: Negative
